## Import libraries

Make sure that R and the following libraries (recommended version ID) are installed on your computer before running this cell:

R.matlab (3.6.2),
xlsx (0.6.1)

In [ ]:
library(R.matlab)
library(xlsx)

## Read data for processing

In [ ]:
# Enter the name of the file to be processed
dataFull <- read.xlsx('SubjectID_ExpType_MM_DD_20YY_protocol.xlsx',1)

# Preview of the data frame
head(dataFull)

## Add some information about the experiment
This information will later be included in the output files and their names

In [ ]:
# Manually enter some information 

dateClock    = "MM-DD-20YY-HH-"       # Date and hour
stayTime     = 1                      # Target duration
subject      = "e.g. subject's name"  # Full ID of the animal
subjectID    = "ML001"                # Short ID of the animal (here: Mouse Lemur 001)
timeout      = 60                     # Trial duration
xArena       = 400                    # X-coordinate of the arena
yArena       = 300                    # Y-coordinate of the arena
radArena     = 300                    # Arena radius
trialCounter = 1                      # Should start at 1              

## Detect indivudal trials in your data frame

In [ ]:
# Creates empty vectors for trial information
trialStart    = c()        # Time [s]
trialStop     = c()        # Time [s]
trialStartT   = c()        # Time [Frame ID]
trialStopT    = c()        # Time [Frame ID]
trialFinished = c()        # 1 = TRUE; 0 = FALSE

# Scans the data frame for trial
for (t in 2:nrow(dataFull)) {
  
  # Changes from phase 1 to 2 = trial initiated
  if (dataFull$Phase[t] == 2 && dataFull$Phase[t-1] == 1)  {
    trialStart   <- c(trialStart, dataFull$Time..s.[t])
    trialStartT  <- c(trialStartT, t)
  }
  # Changes from phase 3 to 0 = trial finished successfully
  if (dataFull$Phase[t] == 0 && dataFull$Phase[t-1] == 3)  {
    trialStop <- c(trialStop, dataFull$Time..s.[t-1])
    trialStopT  <- c(trialStopT, t-1)
    trialFinished <- c(trialFinished, 1)
  }
  # Changes from phase 2 to 0 = trial finished without reward
  if (dataFull$Phase[t] == 0 && dataFull$Phase[t-1] == 2)  {
    trialStop <- c(trialStop, dataFull$Time..s.[t-1])
    trialStopT  <- c(trialStopT, t-1)
    trialFinished <- c(trialFinished, 0)
  }
}

# Removes trials that were unfinished at session end
if (length(trialStart)>length(trialStop)) {
  trialStart   <- trialStart[1:length(trialStop)]
  trialStartT  <- trialStartT[1:length(trialStop)]
}

# Shows the lengths of all vectors (should all be equal)
length(trialStart)
length(trialStop)
length(trialStartT)
length(trialStopT)
length(trialFinished)

# Determines the number of finished trials
no_trials = length(trialFinished)

## Generate output files

This last cell generates an individual MATLAB file for each trial, using the formatting and nomenclature of the original MATLAB script from XYZ et al. 2020 (doi:XYZ). The output files can be fed to MATLAB for final analysis as described in XYZ et al. 2020 (doi:XYZ).

In [ ]:
# Separates the trials and generates an individual output file (= Matlab input) for every trial
for(i in 1:no_trials) { 
  nam <- paste("Trial", i, sep = "")
  assign(nam, dataFull[trialStartT[i]:(trialStopT[i]+1),]) 
  trialData <- eval(parse(text = nam))
  g = nrow(trialData) 
  
  # Adds success marker
  if (trialFinished[i] == 1) {
    trialData$Animal_x[g] = 5000
    trialData$Animal_y[g] = 5000
  }
  
  # Adds fail marker
  else if (trialFinished[i] == 0) {
    trialData$Animal_x[g] = NaN
    trialData$Animal_y[g] = NaN
  }
  
  # Defines values 
  xTarget        = trialData$Target_x[1]
  yTarget        = trialData$Target_y[1]
  radTarget      = trialData$Target_rad[1]
  xSubject       = trialData$Animal_x
  ySubject       = trialData$Animal_y
  xStart         = trialData$Start_x[1]
  yStart         = trialData$Start_y[1]
  radStart       = trialData$Start_rad[1]
  timeStamps     = trialData$Time..s.
  timeStampsNorm = timeStamps-timeStamps[1]
  majAxis     <- rep(1000, length(ySubject))
  minAxis     <- rep(1000, length(ySubject))
  detected    <- rep(1000, length(ySubject))
  orient      <- rep(1000, length(ySubject))
  soundPlayed <- rep(1000, length(ySubject))
  
  # Generates a standardized file name for the current output file
  if (trialCounter < 10) {
    fileName = paste("Pos_",subjectID,"-",dateClock,"0",trialCounter,"-00.mat", sep = "", collapse = NULL)
  }
  else if (trialCounter >= 10) {
    fileName = paste("Pos_",subjectID,"-",dateClock,trialCounter,"-00.mat", sep = "", collapse = NULL)
  }
  trialCounter = trialCounter+1
  
  # Writes a MATLAB file for the current trial
  writeMat(fileName, 
           "wid" = array(c(radTarget), dim = c(1,1)),
           "staytime" = array(c(stayTime), dim = c(1,1)),
           "Cx" = array(c(xArena), dim = c(1,1)),
           "Cy" = array(c(yArena), dim = c(1,1)),
           "Cwid" = array(c(radArena), dim = c(1,1)), 
           "X.plat" = array(c(xStart), dim = c(1,1)),
           "Y.plat" = array(c(yStart), dim = c(1,1)),
           "wid.plat" = array(c(radStart), dim = c(1,1)),
           "timeout" = array(c(timeout), dim = c(1,1)),
           "freq" = array(c(145), dim = c(1,1)), 
           "Xt" = array(c(xTarget), dim = c(1,1)),
           "Yt" = array(c(yTarget), dim = c(1,1)),
           "XYs" = array(c(xSubject,ySubject), dim = c(length(ySubject),2)), 
           "timestamps" = array(c(timeStampsNorm), dim = c(length(ySubject),1)),
           "maj.ax" = array(c(majAxis), dim = c(length(ySubject),1)),
           "min.ax" = array(c(minAxis), dim = c(length(ySubject),1)),
           "detected" = array(c(detected), dim = c(length(ySubject),1)),
           "orient" = array(c(majAxis), dim = c(length(ySubject),1)),
           "sound.played" = array(c(soundPlayed), dim = c(length(ySubject),1)))
}